In [1]:
import json

In [2]:
with open(r"odis_json\64814.json", mode="r", encoding="utf-8") as read_file:
    file_data = json.load(read_file)

In [12]:
# Extract match info directly
match_summary = file_data["info"]  # Keeping the original structure

# Initialize innings summary
innings_summary = []

for inning in file_data["innings"]:
    team = inning["team"]
    total_runs = 0
    total_wickets = 0
    total_extras = 0
    batsmen_stats = {}
    bowlers_stats = {}

    for over in inning["overs"]:
        for delivery in over["deliveries"]:
            batter = delivery["batter"]
            bowler = delivery["bowler"]
            runs = delivery["runs"]["total"]
            extras = delivery["runs"]["extras"]
            total_runs += runs
            total_extras += extras

            # Batting stats
            if batter not in batsmen_stats:
                batsmen_stats[batter] = {"runs": 0, "balls": 0, "strike_rate": 0}
            batsmen_stats[batter]["runs"] += delivery["runs"]["batter"]
            batsmen_stats[batter]["balls"] += 1

            # Bowling stats
            if bowler not in bowlers_stats:
                bowlers_stats[bowler] = {"runs_given": 0, "wickets": 0, "balls": 0}
            bowlers_stats[bowler]["runs_given"] += runs
            bowlers_stats[bowler]["balls"] += 1

            # Wicket stats
            if "wickets" in delivery:
                total_wickets += 1
                if bowler in bowlers_stats:
                    bowlers_stats[bowler]["wickets"] += 1

    # Compute strike rate for batsmen
    for batter, stats in batsmen_stats.items():
        if stats["balls"] > 0:
            stats["strike_rate"] = round((stats["runs"] / stats["balls"]) * 100, 2)

    # Compute economy rate for bowlers
    for bowler, stats in bowlers_stats.items():
        overs_bowled = stats["balls"] / 6
        stats["overs"] = round(overs_bowled, 1)
        if overs_bowled > 0:
            stats["economy"] = round(stats["runs_given"] / overs_bowled, 2)

    # Find top performers
    top_scorer = max(batsmen_stats.items(), key=lambda x: x[1]["runs"], default=(None, {}))
    best_bowler = max(bowlers_stats.items(), key=lambda x: x[1]["wickets"], default=(None, {}))

    innings_summary.append({
        "team": team,
        "total_runs": total_runs,
        "total_wickets": total_wickets,
        "total_extras": total_extras,
        "top_scorer": {"player": top_scorer[0], "runs": top_scorer[1].get("runs", 0)},
        "best_bowler": {"player": best_bowler[0], "wickets": best_bowler[1].get("wickets", 0)},
        "batting_contributions": batsmen_stats,
        "bowling_contributions": bowlers_stats
    })

# Combine match summary and innings summary
match_scorecard = {
    "match_summary": match_summary,  # Keeping "info" as it is
    "innings_summary": innings_summary
}

# Save the summary as JSON
output_file = "match_summary_with_players.json"  # Change this to your desired output path
with open(output_file, "w") as f:
    json.dump(match_scorecard, f, indent=4)

print(f"Match summary JSON file created: {output_file}")


Match summary JSON file created: match_summary_with_players.json


In [11]:
import json
import pandas as pd
from pathlib import Path
from collections import defaultdict

def extract_player_stats_flat(json_path):
    with open(json_path, 'r') as f:
        file_data = json.load(f)

    match_id = Path(json_path).stem
    match_date = file_data["info"]["dates"][0]
    innings = file_data["innings"]

    player_stats = defaultdict(lambda: {
        "match_id": match_id,
        "match_date": match_date,
        "team": None,
        "runs_scored": 0,
        "balls_faced": 0,
        "runs_conceded": 0,
        "balls_bowled": 0,
        "wickets": 0,
        "extras_conceded": 0,
        "innings_batted": None,
        "innings_bowled": None,
    })

    for inning_idx, inning in enumerate(innings, start=1):
        team = inning["team"]
        overs = inning["overs"]

        for over in overs:
            for delivery in over["deliveries"]:
                batter = delivery["batter"]
                bowler = delivery["bowler"]
                total_runs = delivery["runs"]["total"]
                batter_runs = delivery["runs"]["batter"]
                extras = delivery["runs"]["extras"]

                # Batting stats
                b_stats = player_stats[batter]
                b_stats["team"] = team
                b_stats["runs_scored"] += batter_runs
                b_stats["balls_faced"] += 1
                b_stats["innings_batted"] = inning_idx

                # Bowling stats
                bo_stats = player_stats[bowler]
                bo_stats["team"] = [t for t in file_data["info"]["teams"] if t != team][0]
                bo_stats["runs_conceded"] += total_runs
                bo_stats["balls_bowled"] += 1
                bo_stats["extras_conceded"] += extras
                bo_stats["innings_bowled"] = inning_idx

                # Wickets
                if "wickets" in delivery:
                    for w in delivery["wickets"]:
                        if w["kind"] != "run out":
                            bo_stats["wickets"] += 1

    # Convert to DataFrame
    flat_stats = []
    for player, stats in player_stats.items():
        stats["player_name"] = player
        flat_stats.append(stats)

    df = pd.DataFrame(flat_stats)
    return df

In [13]:
df = extract_player_stats_flat("data/raw/extracted/64814.json")
print(df)

# Save as CSV
df.to_csv("data/processed/64814_player_stats.csv", index=False)


   match_id  match_date         team  runs_scored  balls_faced  runs_conceded  \
0     64814  2002-12-29  New Zealand            1            6              0   
1     64814  2002-12-29        India            3            6             40   
2     64814  2002-12-29  New Zealand           76           97              0   
3     64814  2002-12-29        India           11           13             54   
4     64814  2002-12-29  New Zealand           78          127              0   
5     64814  2002-12-29        India            0            0             44   
6     64814  2002-12-29        India           14           17             58   
7     64814  2002-12-29        India            4            5             34   
8     64814  2002-12-29        India            0            3             24   
9     64814  2002-12-29  New Zealand            5            7              0   
10    64814  2002-12-29  New Zealand           34           32              0   
11    64814  2002-12-29  New